# Автоматическая обработка текстов 
## Домашнее задание 1 [10 баллов] до 23:59 15.03.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе 
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]


Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание, с любого сервиса с прогнозом погоды или используя его API.
Примеры сервисов:
* gismeteo.ru, https://www.gismeteo.ru/api/ – Gismeteo, API Gismeteo 
* https://tech.yandex.ru/weather/ – API Yandex.Погоды
* https://sinoptik.com.ru – Sinoptik
* любой другой 

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальная температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

In [80]:
import json
import numpy as np
import pandas as pd
import requests
import datetime
import pymorphy2
import calendar

In [2]:
CITY, CITY_ENG = 'Москвe', 'Moscow'

def get_city():
    return CITY


MIN_TEMPERATURE = 'минимальная температура'
MAX_TEMPERATURE = 'максимальная температура'
WIND_SPEED = 'скорость ветра'
PRECIPITATION_LEVEL = 'уровень осадков'

In [3]:
forecast_url = 'http://api.wunderground.com/api/f80f6ff287475a26/forecast10day/q/RU/' + CITY_ENG + '.json'

In [4]:
forecast_data = requests.get(forecast_url).json()
daily10_forecast_data = forecast_data['forecast']['simpleforecast']['forecastday']

In [33]:
forecast_row_index = [MIN_TEMPERATURE, MAX_TEMPERATURE, WIND_SPEED, PRECIPITATION_LEVEL]
forecast_table = pd.DataFrame(np.zeros(shape=(4, 10), dtype=np.int32), index=forecast_row_index)

dates = []

for i in range(10):
    daily_forecast = daily10_forecast_data[i]

    forecast_table.set_value(forecast_row_index[0], i, daily_forecast['low']['celsius'])
    forecast_table.set_value(forecast_row_index[1], i, daily_forecast['high']['celsius'])
    forecast_table.set_value(forecast_row_index[2], i, daily_forecast['avewind']['kph'])
    
    qpf_daily = daily_forecast['qpf_day']['mm']
    if qpf_daily == None:
        qpf_daily = 0
    forecast_table.set_value(forecast_row_index[3], i, qpf_daily )
    
    day = datetime.date(daily_forecast['date']['year'], daily_forecast['date']['month'], daily_forecast['date']['day'])
    dates.append(day)
    
forecast_table.rename(columns=lambda x: dates[x], inplace=True)

In [34]:
forecast_table

,2018-03-09,2018-03-10,2018-03-11,2018-03-12,2018-03-13,2018-03-14,2018-03-15,2018-03-16,2018-03-17,2018-03-18
минимальная температура,-12,-11,-11,-10,-11,-10,-8,-8,-9,-9
максимальная температура,-4,-5,-4,-3,-3,-3,-2,-1,-2,-2
скорость ветра,14,19,18,14,21,18,16,10,13,14
уровень осадков,0,1,1,0,0,0,0,0,0,0


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на первые 10 дней февраля в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 км/час в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 мм за 7 дней*
    * *Выпадет 10 см снега за ближайшие 7 дней * 


Вместо НазваниеГорода и дней недели подставьте название выбранного вами города и дни недели, используя фунцкии для согласования существительных с предлогами. Используйте функции для согласования числительного с существительным для согласования длительности промежутков времени и слова "день" и чисел  с последующими единицами измерения.

In [35]:
forecast_table.index

Index(['минимальная температура', 'максимальная температура', 'скорость ветра',
       'уровень осадков'],
      dtype='object')

In [48]:
print(MIN_TEMPERATURE,',', MAX_TEMPERATURE,',', WIND_SPEED,',', PRECIPITATION_LEVEL)

минимальная температура , максимальная температура , скорость ветра , уровень осадков


In [44]:
DAYS = forecast_table.columns

In [46]:
DAYS

Index([2018-03-09, 2018-03-10, 2018-03-11, 2018-03-12, 2018-03-13, 2018-03-14,
       2018-03-15, 2018-03-16, 2018-03-17, 2018-03-18],
      dtype='object')

In [59]:
# index - одно из четырех значений, например MIN_TEMPERATURE
# day - число от 0 до 9
def get_value_for_day(index, day):
    return forecast_table.get_value(index, DAYS[day])
    
print(get_value_for_day(WIND_SPEED, 9))

14


In [88]:
def get_weekday_in_russian(day):
    if day == 0:
        return 'понедельник'
    elif day == 1:
        return 'вторник'
    elif day == 2:
        return 'среда'
    elif day == 3:
        return 'четверг'
    elif day == 4:
        return 'пятница'
    elif day == 5:
        return 'суббота'
    elif day == 6:
        return 'воскресенье'
    assert(false)
    
print(get_weekday_in_russian(6))

воскресенье


In [113]:
def make_first_letter_uppercase(word):
    return word[0].upper() + word[1:]

print(make_first_letter_uppercase('счастье'))

Счастье


In [131]:
morph = pymorphy2.MorphAnalyzer()

In [160]:
def get_forecast_type_1(day1, day2):
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word

    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    min_temp1 = get_value_for_day(MIN_TEMPERATURE, day1)
    max_temp1 = get_value_for_day(MAX_TEMPERATURE, day1)
    avg_temp1 = (max_temp1 + min_temp1) // 2.
    
    min_temp2 = get_value_for_day(MIN_TEMPERATURE, day2)
    max_temp2 = get_value_for_day(MAX_TEMPERATURE, day2)
    avg_temp2 = (max_temp2 + min_temp2) // 2.
    
    delta_temp = avg_temp2 - avg_temp1
        
    action = 'потеплеет'
    if delta_temp < 0:
        action = 'похолодает'
        delta_temp *= -1
        
    if delta_temp == 0:
        action = 'температура не изменится'
        TEMPLATE = "В {} в {} {} по сравнению c {}"
        return TEMPLATE.format(day2_word, city_word, action, day1_word)
    
    units = 'градус'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_temp).word
        
    TEMPLATE = "В {} в {} {} на {} {} по сравнению c {}"
    return TEMPLATE.format(day2_word, city_word, action, delta_temp, correct_units, day1_word)
    
print(get_forecast_type_1(0, 3))

В понедельник в Москве потеплеет на 1.0 градус по сравнению c пятницей


In [161]:
# код нужно тестировать
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_1(i, j))

В субботу в Москве температура не изменится по сравнению c пятницей
В воскресенье в Москве температура не изменится по сравнению c пятницей
В понедельник в Москве потеплеет на 1.0 градус по сравнению c пятницей
В вторник в Москве потеплеет на 1.0 градус по сравнению c пятницей
В среду в Москве потеплеет на 1.0 градус по сравнению c пятницей
В четверг в Москве потеплеет на 3.0 градуса по сравнению c пятницей
В воскресенье в Москве температура не изменится по сравнению c субботой
В понедельник в Москве потеплеет на 1.0 градус по сравнению c субботой
В вторник в Москве потеплеет на 1.0 градус по сравнению c субботой
В среду в Москве потеплеет на 1.0 градус по сравнению c субботой
В четверг в Москве потеплеет на 3.0 градуса по сравнению c субботой
В понедельник в Москве потеплеет на 1.0 градус по сравнению c воскресеньем
В вторник в Москве потеплеет на 1.0 градус по сравнению c воскресеньем
В среду в Москве потеплеет на 1.0 градус по сравнению c воскресеньем
В четверг в Москве потеплеет на

In [162]:
print(get_forecast_type_1(6, 5))

В среду в Москве похолодает на 2.0 градуса по сравнению c четвергом


In [151]:
# "Скорость ветра изменится на X единиц в день1 по сравнению с день2." плохой пример, нет города и "изменится" - плохое слово

In [185]:
def get_forecast_type_2(day1, day2):
    wind_speed_word = make_first_letter_uppercase(WIND_SPEED)
    
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word

    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    wind_speed1 = get_value_for_day(WIND_SPEED, day1)
    wind_speed2 = get_value_for_day(WIND_SPEED, day2)
    
    delta_wind_speed = wind_speed2 - wind_speed1
    
    action = 'увеличится'
    if delta_wind_speed < 0:
        delta_wind_speed *= -1
        action = 'уменьшится'
        
    if delta_wind_speed == 0:
        action = 'не изменится'
        TEMPLATE = "{} в {} в {} {} по сравнению c {}"
        return TEMPLATE.format(wind_speed_word, day2_word, city_word, action, day1_word)
    
    units = 'единица'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_wind_speed)
    
    # костыль для ошибки при значении delta_wind_speed = 1
    if delta_wind_speed == 1:
        correct_units = correct_units.inflect({'sing', 'accs'})
    
    correct_units_word = correct_units.word
    
    TEMPLATE = "{} в {} в {} {} на {} {} по сравнению c {}"
    return TEMPLATE.format(wind_speed_word, day2_word, city_word, action, delta_wind_speed, correct_units_word, day1_word)
    
print(get_forecast_type_2(1, 2))

Скорость ветра в воскресенье в Москве уменьшится на 1 единицу по сравнению c субботой


In [187]:
# код нужно тестировать
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_2(i, j))

Скорость ветра в субботу в Москве увеличится на 5 единиц по сравнению c пятницей
Скорость ветра в воскресенье в Москве увеличится на 4 единицы по сравнению c пятницей
Скорость ветра в понедельник в Москве не изменится по сравнению c пятницей
Скорость ветра в вторник в Москве увеличится на 7 единиц по сравнению c пятницей
Скорость ветра в среду в Москве увеличится на 4 единицы по сравнению c пятницей
Скорость ветра в четверг в Москве увеличится на 2 единицы по сравнению c пятницей
Скорость ветра в воскресенье в Москве уменьшится на 1 единицу по сравнению c субботой
Скорость ветра в понедельник в Москве уменьшится на 5 единиц по сравнению c субботой
Скорость ветра в вторник в Москве увеличится на 2 единицы по сравнению c субботой
Скорость ветра в среду в Москве уменьшится на 1 единицу по сравнению c субботой
Скорость ветра в четверг в Москве уменьшится на 3 единицы по сравнению c субботой
Скорость ветра в понедельник в Москве уменьшится на 4 единицы по сравнению c воскресеньем
Скорость в

In [ ]:
"Уровень осадков повысится / понизится на X единиц за Y дней."

In [201]:
def get_forecast_type_3(day1, day2):
    precipitation_level_word = make_first_letter_uppercase(PRECIPITATION_LEVEL)
    
    day1_word = morph.parse(get_weekday_in_russian(DAYS[day1].weekday()))[0].inflect({'ablt'}).word
    day2_word = morph.parse(get_weekday_in_russian(DAYS[day2].weekday()))[0].inflect({'accs'}).word
    
    delta_days = day2 - day1 + 1
    time_unit = 'день'
    time_unit_word = morph.parse(time_unit)[0].make_agree_with_number(delta_days).word
    
    city_word = make_first_letter_uppercase(morph.parse(get_city())[0].inflect({'datv'}).word)
    
    precipitation_level1 = get_value_for_day(PRECIPITATION_LEVEL, day1)
    precipitation_level2 = get_value_for_day(PRECIPITATION_LEVEL, day2)
    
    delta_precipitation_level = precipitation_level2 - precipitation_level1
    
    action = 'повысится'
    if delta_precipitation_level < 0:
        delta_precipitation_level *= -1
        action = 'понизится'
        
    if delta_precipitation_level == 0:
        action = 'не изменится'
        TEMPLATE = "{} в {} {} за {} {}"
        return TEMPLATE.format(precipitation_level_word, city_word, action, delta_days, time_unit_word )
    
    units = 'единица'
    correct_units = morph.parse(units)[0].make_agree_with_number(delta_precipitation_level)
    
    # костыль для ошибки при значении delta_wind_speed = 1
    if delta_precipitation_level == 1:
        correct_units = correct_units.inflect({'sing', 'accs'})
    
    correct_units_word = correct_units.word
    
    TEMPLATE = "{} в {} {} на {} {} за {} {}"
    return TEMPLATE.format(precipitation_level_word, 
                           city_word, 
                           action,
                           delta_precipitation_level,
                           correct_units_word,
                           delta_days,
                           time_unit_word)
    
print(get_forecast_type_3(1, 2))

Уровень осадков в Москве не изменится за 2 дня


In [202]:
# код нужно тестировать
for i in range(0, 7):
    for j in range(i+1,7):
        print(get_forecast_type_3(i, j))

Уровень осадков в Москве повысится на 1 единицу за 2 дня
Уровень осадков в Москве повысится на 1 единицу за 3 дня
Уровень осадков в Москве не изменится за 4 дня
Уровень осадков в Москве не изменится за 5 дней
Уровень осадков в Москве не изменится за 6 дней
Уровень осадков в Москве не изменится за 7 дней
Уровень осадков в Москве не изменится за 2 дня
Уровень осадков в Москве понизится на 1 единицу за 3 дня
Уровень осадков в Москве понизится на 1 единицу за 4 дня
Уровень осадков в Москве понизится на 1 единицу за 5 дней
Уровень осадков в Москве понизится на 1 единицу за 6 дней
Уровень осадков в Москве понизится на 1 единицу за 2 дня
Уровень осадков в Москве понизится на 1 единицу за 3 дня
Уровень осадков в Москве понизится на 1 единицу за 4 дня
Уровень осадков в Москве понизится на 1 единицу за 5 дней
Уровень осадков в Москве не изменится за 2 дня
Уровень осадков в Москве не изменится за 3 дня
Уровень осадков в Москве не изменится за 4 дня
Уровень осадков в Москве не изменится за 2 дня
У

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
word = 'яблоко'
parsed_word = morph.parse(word)[0]
print(1, parsed_word.make_agree_with_number(1).word) # согласование слова с числительным 1
print(2, parsed_word.make_agree_with_number(2).word) # согласование слова с числительным 2
print(5, parsed_word.make_agree_with_number(5).word) # согласование слова с числительным 5

1 яблоко
2 яблока
5 яблок


In [4]:
print(parsed_word.inflect({'gent'}).word) # слово в родительном падеже

яблока


Ниже приведен пример решения аналогичной задачи генерации текстов: генератор отчета о том, сколько и каких фруктов съел Вася.

In [5]:
def eating(n, fruit):
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join(['Вася съел', str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [6]:
eating(5, 'яблоко')

'Вася съел 5 яблок .'

In [7]:
eating(4, 'груша')

'Вася съел 4 груши .'

Пример более сложного генератора: фрукты могут есть не только мальчики, но и девочки, то есть, нужно не только согласовать числительное с существительным, но и поставить глагол в нужную форму.

In [8]:
def eating(name, n, fruit):
    name_tag = morph.parse(name)[0].tag
    if 'masc' in name_tag:
        gender = 'masc'
    if 'femn' in name_tag:
        gender = 'femn'
    verb = morph.parse("съесть")[0].inflect({'perf', gender,'sing','past','indc'}).word
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join([name, verb, str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [9]:
eating('Маша', 3, 'вишня')

'Маша съела 3 вишни .'

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 15.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку. Плагиат наказывается нулём баллов за задание.

При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям – Антону Емельянову. 

Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.


### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт 0pobDsj (для всех групп). 


 У вас появится курс “МФТИ > Автоматическая обработка текстов (2018)” в разделе “Посещает курсы”.
* Перейдите по ссылке “МФТИ > Автоматическая обработка текстов (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/325